# FM 9578 Assignment 2.1 - Week 3
Unit 1: Computational Finance -week 3.

In [95]:
import yfinance as yf
import pandas as pd
import numpy as np

# Download data
tsx = yf.download("^GSPTSE", start='2007-01-01', end='2012-01-02') #add a day because python cuts off last entry

# Q1 
# Fill missing weekdays (business days) with previous value
tsx = tsx.asfreq('B').ffill()

# Add a day of the week column
tsx['Day'] = tsx.index.strftime('%a')

[*********************100%***********************]  1 of 1 completed


In [96]:
# Q2
# Calculate 3-day MA
tsx['MA3'] = tsx['Adj Close'].rolling(3).mean()
weekly_mas = [] #empty list to store MAs


for i in range(0, len(tsx), 5):  # Loop through dataset week by week (in chunks of 5 days)
    week_data = tsx.iloc[i:i+5]  
    if len(week_data) == 5:
        #Get MA from the last day in MA for each week, i.e. Wed, gives the MA for (Mon,Tue,Wed)
        #since data starts on a Tuesday, the first MA will be NaN
        weekly_mas.append(week_data.loc[week_data['Day'] == 'Wed', 'MA3'].values[0])  # MA(Mon, Tue, Wed)
        weekly_mas.append(week_data.loc[week_data['Day'] == 'Thu', 'MA3'].values[0])  # MA(Tue, Wed, Thu)
        weekly_mas.append(week_data.loc[week_data['Day'] == 'Fri', 'MA3'].values[0])  # MA(Wed, Thu, Fri)

# Create DataFrame for results
weekly_df = pd.DataFrame({
    'Week': np.repeat(range(1, len(weekly_mas) // 3 + 1), 3), #repeat each week 3 times bcuz 3 MA per week
    'MA': weekly_mas
})

weekly_df

,Week,MA
0,1,NaN
1,1,12726.233398
2,1,12577.666667
3,2,12492.899740
4,2,12481.833333
...,...,...
775,259,11602.466797
776,259,11560.966797
777,260,11670.033529
778,260,11782.300130


In [97]:
# Q3 
# Make a series of log returns for the weekly series in Q2
weekly_df['Log_Returns'] = np.log(weekly_df['MA'] / weekly_df['MA'].shift(1))
#weekly_df.dropna(inplace=True)
weekly_df

,Week,MA,Log_Returns
0,1,NaN,NaN
1,1,12726.233398,NaN
2,1,12577.666667,-0.011743
3,2,12492.899740,-0.006762
4,2,12481.833333,-0.000886
...,...,...,...
775,259,11602.466797,-0.011526
776,259,11560.966797,-0.003583
777,260,11670.033529,0.009390
778,260,11782.300130,0.009574


In [102]:
# Q4 
# Compute mean and standard deviation of log returns from Q3
mean_rt = weekly_df['Log_Returns'].mean()
std_rt = weekly_df['Log_Returns'].std()
print(f"Mean Log Return: {100*mean_rt:.4f}%")
print(f"Std Dev Log Return: {100*std_rt:.4f}%")

Mean Log Return: -0.0091%
Std Dev Log Return: 1.3209%


In [104]:
# Q5: Find the annualized values for the mean and standard deviation (drift and volatility).
# Annualized drift and volatility
annual_drift = mean_rt * 52 * 3 #52 weeks per year, 3 returns per week
annual_vol = std_rt * np.sqrt(52*3)

print(f"Annual Drift: {100*annual_drift:.4f}%")
print(f"Annual Volatility: {100*annual_vol:.4f}%")

Annual Drift: -1.4266%
Annual Volatility: 16.4975%


# BONUS

In [100]:
#Pricing an option using the binomial tree model with risk-neutral approach
def binomial_tree_option_pricing(S, K, T, r, u, d, N, option_type='put'):

    # Calculate time step
    dt = T / N
    
    # Calculate risk-neutral probability
    q = (np.exp(r * dt) - d) / (u - d)
    
    # Initialize asset prices at maturity
    asset_prices = np.zeros(N + 1)
    for i in range(N + 1):
        asset_prices[i] = S * (u ** (N - i)) * (d ** i)
    
    # Initialize option values at maturity
    option_values = np.zeros(N + 1)
    if option_type == 'call':
        option_values = np.maximum(0, asset_prices - K)
    elif option_type == 'put':
        option_values = np.maximum(0, K - asset_prices)
    
    # Step backwards through the tree
    for j in range(N - 1, -1, -1):
        for i in range(j + 1):
            option_values[i] = np.exp(-r * dt) * (q * option_values[i] + (1 - q) * option_values[i + 1])
    
    return option_values[0]

# Parameters
S = 80      # Initial stock price
K = 80      # Strike price
T = 4/12    # Time to maturity in years
r = 0.05    # Risk-free interest rate
u = 85/80   # Up factor
d = 75/80   # Down factor
N = 1       # Number of steps in the binomial tree (one-step model)

# Calculate call option price
put_option_price = binomial_tree_option_pricing(S, K, T, r, u, d, N, option_type='put')
print(f"The price of the 4-month European Put option is: ${put_option_price:.2f}")

The price of the 4-month European Put option is: $1.80
